### setup

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import requests
import pageviewapi
import datetime as dt
from bs4 import BeautifulSoup
import altair as alt
import altair_grid as altgrid

alt.themes.register("grid", altgrid.theme)
alt.themes.enable("grid")

import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [3]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [4]:
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="PU52xuCQowgE5BFprvdnpFXzd5Ql9ISOSLBk58eNR2ykPztw0yC5fpqLTfsZdhyR"
)

In [5]:
# scope = ['https://spreadsheets.google.com/feeds']
# credentials = ServiceAccountCredentials.from_json_keyfile_name('jupyter-integration-349314-25735d35924b.json', scope)
# gc = gspread.authorize(credentials)

### get data

In [6]:
# find variable for health insurance coverage under age 19
# S2701_C01_002E Estimate!!Total!!Civilian noninstitutionalized population!!AGE!!Under 6 years
# S2701_C04_002E Estimate!!Uninsured!!Civilian noninstitutionalized population!!AGE!!Under 6 years
# S2701_C05_002E Estimate!!Percent Uninsured!!Civilian noninstitutionalized population!!AGE!!Under 6 years
# S2701_C01_003E Estimate!!Total!!Civilian noninstitutionalized population!!AGE!!6 to 18 years
# S2701_C04_003E Estimate!!Uninsured!!Civilian noninstitutionalized population!!AGE!!6 to 18 years
# S2701_C05_003E Estimate!!Percent Uninsured!!Civilian noninstitutionalized population!!AGE!!6 to 18 years
variables = [
    "S2701_C01_002E",
    "S2701_C04_002E",
    "S2701_C05_002E",
    "S2701_C01_003E",
    "S2701_C04_003E",
    "S2701_C05_003E",
]
years = [2015, 2016, 2017, 2018, 2019, 2021]

In [20]:
frames = []

for year in years:
    for variable in variables:
        frames.append(
            pd.read_json(
                "https://api.census.gov/data/"
                + str(year)
                + "/acs/acs1/subject?get="
                + variable
                + "&for=us:*&key=aa5d6d0fefa3172a9f363636440111d6584a0ac4"
            ).assign(year=year, variable=variable)
        )

In [19]:
frames[0]

,0,1,year
0,S2701_C01_002E,us,2015
1,73491931,1,2015


In [8]:
years = []
values = []
variables = []
for frame in frames:
    years.append(frame["year"].iloc[1])
    values.append(frame.loc[1].iloc[0])
    variables.append(frame.loc[0].iloc[0])

In [9]:
df = pd.DataFrame({"year": years, "value": values, "variable": variables})

In [10]:
df["value"] = df["value"].astype(float)

In [11]:
df_wide = df.pivot(index="year", columns="variable", values="value").reset_index()

In [12]:
df_wide["uninsured_under19"] = df_wide["S2701_C04_002E"] + df_wide["S2701_C04_003E"]
df_wide["pop_under19"] = df_wide["S2701_C01_002E"] + df_wide["S2701_C01_003E"]

In [13]:
df_wide["pct_uninsured_under19"] = 100 * (
    df_wide["uninsured_under19"] / df_wide["pop_under19"]
)

In [14]:
alt.Chart(df_wide).mark_line().encode(x="year:O", y="pct_uninsured_under19")

alt.Chart(...)

In [15]:
df_slim = df_wide[["year", "pct_uninsured_under19"]]

In [16]:
df_slim["series_15-19"] = [4.612646, 4.301608, 5.026953, 5.211412, 5.656209, "NaN"]
df_slim["series_19-21"] = ["NaN", "NaN", "NaN", "NaN", 5.656209, 5.350971]

/var/folders/gg/6wnjn6cd1qlfst8mkw4g4xqc0000gn/T/ipykernel_36610/2346405737.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_slim["series_15-19"] = [4.612646, 4.301608, 5.026953, 5.211412, 5.656209, "NaN"]
/var/folders/gg/6wnjn6cd1qlfst8mkw4g4xqc0000gn/T/ipykernel_36610/2346405737.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_slim["series_19-21"] = ["NaN", "NaN", "NaN", "NaN", 5.656209, 5.350971]


In [17]:
# dw.create_chart("Children uninsured over time", chart_type="d3-lines", data=df_slim)
chart_id = "bJmTu"